In [1]:
import pandas as pd
# Display columns
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)
pd.options.display.width=None

import os
os.chdir("/usr/src/app")

# Tournaments from ATP

In [19]:
PATH_TOURNAMENTS = "deepsetstats/dataset/tournaments/parquet/tournaments.parquet"
df = pd.read_parquet(PATH_TOURNAMENTS, engine="pyarrow")


# Create dicts to create in tournaments/constants.py the mapping of the real names of the tournaments
# that will be later used by tournaments/get_tournaments_youtube_naming.py
#big = df.query("level in ('grandslam', '1000')")
#medium = df.query("level in ('500')").sort_values("date_start").set_index('city')['name'].to_dict()
#small = df.query("level in ('250')").sort_values("date_start").set_index('city')['name'].to_dict()

# Tournaments namings Tennis TV

In [25]:
import difflib
from fuzzywuzzy import fuzz, process
from copy import copy
import string
import regex
import re

In [28]:
PATH_TOURNAMENTS_NAMING = "deepsetstats/dataset/tournaments/parquet/tournaments_tennistv.parquet"
df_tennistv = pd.read_parquet(PATH_TOURNAMENTS_NAMING, engine="pyarrow")

In [39]:
df_tennistv

,level,city,country,court_type,month,name,date_start,date_end,tournament_name,tournament_id
0,grandslam,Melbourne,Australia,Outdoor Hard,Jan,Australian Open,2023-01-16,2023-01-29,Australian Open,0
1,grandslam,Paris,France,Outdoor Clay,May,Roland Garros,2023-05-28,2023-06-11,Roland Garros,1
2,grandslam,London,Great Britain,Outdoor Grass,Jul,Wimbledon,2023-07-03,2023-07-16,Wimbledon,2
3,grandslam,"New York, NY",U.S.A.,Outdoor Hard,Aug,US Open,2023-08-28,2023-09-10,US Open,3
4,1000,"Indian Wells, CA",U.S.A.,Outdoor Hard,Mar,BNP Paribas Open,2023-03-08,2023-03-19,Indian Wells,4
5,1000,"Miami, FL",U.S.A.,Outdoor Hard,Mar,Miami Open presented by Itau,2023-03-22,2023-04-02,Miami,5
6,1000,Monte-Carlo,Monaco,Outdoor Clay,Apr,Rolex Monte-Carlo Masters,2023-04-09,2023-04-16,Monte Carlo,6
7,1000,Madrid,Spain,Outdoor Clay,Apr,Mutua Madrid Open,2023-04-26,2023-05-07,Madrid,7
8,1000,Rome,Italy,Outdoor Clay,May,Internazionali BNL d'Italia,2023-05-10,2023-05-21,Rome,8
9,1000,Toronto,Canada,Outdoor Hard,Aug,National Bank Open Presented by Rogers,2023-08-07,2023-08-13,Toronto,9


In [30]:
# Set of player names
tournament_names = df_tennistv["tournament_name"].tolist()
tournament_ids = df_tennistv["tournament_id"].tolist()

In [31]:
# Define the translation table
exclude_punctuation = string.punctuation
translator = str.maketrans(exclude_punctuation, ' ' * len(exclude_punctuation))

In [36]:
def extract_tournament_title(raw_string, tournament_names, tournament_ids):
    query_string = raw_string.translate(translator)

    # Replace consecutive spaces with single spaces
    query_string = re.sub(r'\s+', ' ', query_string)
    
    final_tournament_id = None
    final_tournament_name = None
    
    for tournament_name, tournament_id in zip(
            tournament_names, tournament_ids
        ):
        # Remove punctuation and replace with spaces
        tournament_name = tournament_name.translate(translator)
        permitted_errors = 1
        match_tournament_name = regex.search(fr'(\s)({tournament_name})(\s){{e<={permitted_errors}}}', query_string)
        # If full name is there
        if match_tournament_name:
            # print(f"Matched tournament: {tournament_name}, id: {tournament_id}")
            final_tournament_id = tournament_id
            final_tournament_name = tournament_name
            return final_tournament_id, final_tournament_name

In [38]:
raw_string = 'Dimitrov Encounters Thiem Van De Zandschulp Faces Griekspoor Winston Salem Highlights Day 3'
tour_id, tour_name = extract_tournament_title(raw_string, tournament_names, tournament_ids)
print(f"Tournament {tour_id}: {tour_name}")

Matched tournament: Winston Salem, id: 56
Tournament 56: Winston Salem


In [46]:
tournament_name

'Winston Salem'

# Extract Tournament Title development